---
## Hausaufgabe 4 - Metaheuristiken
---

### Achtung: Bitte überprüfen Sie vor Abgabe der Hausaufgabe, ob das Notebook richtig gespeichert wurde. 
Die Speicherung von Notebooks funktioniert über das "Disketten"-Symbol im Notebook oder über die Shortcuts Strg + S (Win) und CMD + S (MAC).

---
## Bewertung
---

#### Erreichbare Punkte: 25
---
#### Erreichte Punkte:
---
---
Aufgabe 1: <br>
<br>
Aufgabe 2: <br>
<br>
Aufgabe 3: <br>

#### Aufgabe 1 - Zeitmessung (4 Punkte)

Die Performance von Metaheuristiken wird neben der Lösungsqualität auch häufig über die benötigte Laufzeit gemessen. Implementieren Sie eine Zeitmessung für die Durchführung des Solver unter Benutzung des `default_timer` der Bibliothek `timeit`. Der Startpunkt der Messung ist direkt vor `RunLocalSearch` und der Endpunkt nach der Ausgabe der besten gefundenen Lösung. 

Ergänzen Sie den gegebenen Quellcode um die Zeitmessung.


In [2]:
from Solver import *
import timeit


data = InputData("VFR20_10_1_SIST.json") 
solver = Solver(data, 1008)

localSearch = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
iteratedGreedy = IteratedGreedy(
    inputData = data, 
    numberJobsToRemove = 2, 
    baseTemperature = 0.8, 
    maxIterations = 10,
  
    localSearchAlgorithm = localSearch)

startTime = timeit.default_timer()
solver.RunLocalSearch(
    constructiveSolutionMethod='NEH',
    algorithm=iteratedGreedy)

endTime = timeit.default_timer()
print(f'Runtime: {endTime - startTime} seconds')

Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
New best solution in iteration 7: The permutation [14, 12, 8, 10, 2, 13, 18, 16, 0, 1, 11, 19, 7, 3, 6, 9, 17, 5, 15, 4] results in a Makespan of 1569
Best found Solution.
The permutation [14, 12, 8, 10, 2, 13, 18, 16, 0, 1, 11, 19, 7, 3, 6, 9, 17, 5, 15, 4] results in a Makespan of 1569
Runtime: 3.766397699999999 seconds


Mögliches erwartetes Ergebnis (Runtime kann abweichen):

    Generating an initial solution according to NEH.
    Constructive solution found.
    The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
    New best solution in iteration 7: The permutation [14, 12, 8, 10, 2, 13, 18, 16, 0, 1, 11, 19, 7, 3, 6, 9, 17, 5, 15, 4] results in a Makespan of 1569
    Best found Solution.
    The permutation [14, 12, 8, 10, 2, 13, 18, 16, 0, 1, 11, 19, 7, 3, 6, 9, 17, 5, 15, 4] results in a Makespan of 1569
    Runtime: 3.318953300000004 seconds

#### Aufgabe 2: Stoppkriterium (6 Punkte)

Neben der gesamten Anzahl an Iterationen wird häufig auch die Anzahl an Iterationen ohne Verbesserung der bisher besten gefunden Lösung als Stoppkriterium gewählt. Ergänzen Sie die Klasse `IteratedGreedy` in `ImprovementAlgorithm.py` so um einen Member `MaxIterationsWithoutImprovement`, dass dessen Wert analog zu `MaxIterations` im Konstruktur übergeben werden kann. 

Die While-Schleife in der Funktion `Run` soll abbrechen, wenn eins der beiden Stoppkriterien erreicht ist.

**Tipp:**
<details>

Neben der Zählvariable `i` wird eine weitere Zählvariable benötigt. Diese wird erhöht, wenn keine beste Lösung gefunden wird und zurückgesetzt, wenn eine neue beste Lösung gefunden wird.
</details>

Ergänzen Sie untenstehenden Quellcode um Ihre neue Variante von Iterated Greedy. Nehmen Sie für Parameter `MaxIterationsWithoutImprovement` den Wert 2 an und für alle anderen die Werte aus Aufgabe 1. 

In [2]:
""" Angepasste Sätze habe ich mit ######## gekenzeichnet """
class IteratedGreedy(ImprovementAlgorithm):
    def __init__(self, inputData, numberJobsToRemove, baseTemperature, maxIterations, maxIterationsWithoutImprovement , localSearchAlgorithm = None):
        super().__init__(inputData)

        self.NumberJobsToRemove = numberJobsToRemove
        self.BaseTemperature = baseTemperature
        self.MaxIterations = maxIterations

        self.MaxIterationsWithoutImprovement = maxIterationsWithoutImprovement ###############

        if localSearchAlgorithm is not None:
            self.LocalSearchAlgorithm = localSearchAlgorithm
        else:
            self.LocalSearchAlgorithm = IterativeImprovement(self.InputData, neighborhoodTypes=[]) # IterativeImprovement without a neighborhood does not modify the solution
    
    def Initialize(self, evaluationLogic, solutionPool, rng):
        self.EvaluationLogic = evaluationLogic
        self.SolutionPool = solutionPool
        self.RNG = rng

        self.LocalSearchAlgorithm.Initialize(self.EvaluationLogic, self.SolutionPool)
    
    def Destruction(self, currentSolution):
        removedJobs = self.RNG.choice(self.InputData.n, size=self.NumberJobsToRemove, replace = False).tolist()

        partialPermutation = [i for i in currentSolution.Permutation if i not in removedJobs]

        return removedJobs, partialPermutation

    def Construction(self, removedJobs, permutation):
        completeSolution = Solution(self.InputData.InputJobs, permutation)
        for i in removedJobs:
            self.EvaluationLogic.DetermineBestInsertionAccelerated(completeSolution, i)

        return completeSolution

    def AcceptWorseSolution(self, currentObjectiveValue, newObjectiveValue):
        randomNumber = self.RNG.random()

        totalProcessingTime = sum(x.ProcessingTime(i) for x in self.InputData.InputJobs for i in range(len(x.Operations)))
        Temperature = self.BaseTemperature * totalProcessingTime / (self.InputData.n * self.InputData.m * 10)
        probability = math.exp(-(newObjectiveValue - currentObjectiveValue) / Temperature)
        
        return randomNumber <= probability

    def Run(self, currentSolution):
        currentSolution = self.LocalSearchAlgorithm.Run(currentSolution)

        currentBest = self.SolutionPool.GetLowestMakespanSolution().Makespan
        iteration = 0
        iteration_j = 0 ########################

        while(iteration < self.MaxIterations and iteration_j < self.MaxIterationsWithoutImprovement):   ###### 
            removedJobs, partialPermutation = self.Destruction(currentSolution)
            newSolution = self.Construction(removedJobs, partialPermutation)

            newSolution = self.LocalSearchAlgorithm.Run(newSolution)
            
            if newSolution.Makespan < currentSolution.Makespan:
                currentSolution = newSolution

                if newSolution.Makespan < currentBest:
                    print(f'New best solution in iteration {iteration}: {currentSolution}')
                    self.SolutionPool.AddSolution(currentSolution)
                    currentBest = newSolution.Makespan
                    iteration_j = 0 ################

            elif self.AcceptWorseSolution(currentSolution.Makespan, newSolution.Makespan):
                currentSolution = newSolution
                iteration_j += 1    #################

            iteration += 1

        return self.SolutionPool.GetLowestMakespanSolution()

In [1]:
from Solver import *

data = InputData("VFR20_10_1_SIST.json") 
solver = Solver(data, 1008)


localSearch = IterativeImprovement(data, 'BestImprovement', ['Insertion'])
iteratedGreedy = IteratedGreedy(    
    inputData = data,    ##########      ######ffff
    numberJobsToRemove = 2, 
    baseTemperature = 0.8, 
    maxIterations = 10,
    maxIterationsWithoutImprovement = 2,
    localSearchAlgorithm = localSearch)

solver.RunLocalSearch(
    constructiveSolutionMethod='NEH',
    algorithm=iteratedGreedy)


Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
Best found Solution.
The permutation [14, 5, 12, 10, 17, 8, 2, 13, 1, 18, 0, 16, 11, 19, 7, 3, 9, 6, 15, 4] results in a Makespan of 1595


Mögliches erwartetes Ergebis:
  
    Generating an initial solution according to NEH.
    Constructive solution found.
    The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
    Best found Solution.
    The permutation [14, 5, 12, 10, 17, 8, 2, 13, 1, 18, 0, 16, 11, 19, 7, 3, 9, 6, 15, 4] results in a Makespan of 1595

#### Aufgabe 3: Rechenstudie (15 Punkte)

Rechenstudien werden zur Bewertung von Algorithmen und zum Finden guter Parameterkombinationen durchgeführt. Im Folgenden sollen Sie eine kleine Rechenstudie mit der Instanz `VFR20_10_1_SIST.json` durchführen. Testen Sie __alle__ Kombinationen (vollfaktorieller Versuchsplan) der Parameter:

- Lokale Suche mit `BestImprovement`: keine lokale Suche (keine Nachbarschaft), Insertion-Nachbarschaft, TaillardInsertion-Nachbarschaft 
- numberJobsToRemove: 2, 3, 4
- baseTemperature: 0.5, 1
- maxIterations 1, 10
- maxIterationsWithoutImprovement soll keine Rolle spielen (bspw. einfach Wert > maxIterations).



Da Iterated Greedy vom Zufall beeinflusst ist, führen Sie __jeweils__ 3 Iterationen mit unterschiedlichen Seeds für __jede__ Parameterkombinationen durch. Mit `iteration` als Laufvariable für die aktuelle Iteration, soll der Seed wie folgt berechnet werden:
seed = `numberJobsToRemove`  x  `maxIterations`  x  `iteration`

Speichern Sie für jede Iteration die folgenden Daten in einem Dictionary (https://docs.python.org/3/tutorial/datastructures.html#dictionaries):

- Iteration (Key: Iteration)
- Jeden Parameter, außer maxIterationsWithoutImprovement (siehe Aufzählung oben) als einzelner Eintrag (Keys: LocalSearch, NumberJobsToRemove, BaseTemperature, MaxIterations)
- Seed (Key: Seed)
- Makespan(Key: Makespan)
- Laufzeit (Key: Runtime; vgl. Aufgabe 1)

Fügen Sie dann das Dictionary einer Liste `rows` hinzu. Erzeugen Sie nach dem alle Versuche beendet sind aus `rows` einen pandas data frame `results` (https://stackoverflow.com/a/17496530). Bilden Sie mit der Funktion `groupby` den Mittelwert (https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.mean.html) von `Makespan` und `Runtime` für die 4 Paramter `LocalSearch`, `NumberJobsToRemove`, `BaseTemperature`, `MaxIterations`.



In [1]:
import pandas as pd
import timeit
from Solver import *
data = InputData("VFR20_10_1_SIST.json") 
# iteration = 0
list_numberJobsToRemove = [2, 3, 4]
list_baseTemperature = [0.5, 1]
list_maxIterations = [1, 10]

dict_iteration = {}
dict_iteration['Iteration'] = []
dict_iteration['LocalSearch'] = []
dict_iteration['NumberJobsToRemove'] =[]
dict_iteration['baseTemperature'] = []
dict_iteration['MaxIteration'] = []

dict_iteration['Makespan'] = []
dict_iteration['Runtime'] = []

for LS_Algorithmus in [['Insertion'], None, ['TaillardInsertion']]:
    if LS_Algorithmus is not None:
        for numberJobsToRemove in list_numberJobsToRemove:
            for baseTemperature in list_baseTemperature:
                for maxIteration in list_maxIterations:
                   
                    iteration = 0
                    while iteration < 3:
                        dict_iteration['Iteration'].append(iteration)
                        dict_iteration['LocalSearch'].append(*LS_Algorithmus)
                        dict_iteration['NumberJobsToRemove'].append(numberJobsToRemove)
                        dict_iteration['baseTemperature'].append(baseTemperature)
                        dict_iteration['MaxIteration'].append(maxIteration)

                        seed = numberJobsToRemove*maxIteration*iteration
                        solver = Solver(data, seed)
                        localSearch = IterativeImprovement(data, 'BestImprovement', LS_Algorithmus)

                        localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool, solver.RNG)

                        iteratedGreedy = IteratedGreedy(
                            data,
                            numberJobsToRemove = numberJobsToRemove,
                            baseTemperature = baseTemperature,
                            maxIterations = maxIteration,
                            localSearchAlgorithm = localSearch)
                            
                        bestInitialSol = solver.ConstructionPhase('NEH')
                        

                        startTime = timeit.default_timer()
                       

                        iteratedGreedy.Initialize(solver.EvaluationLogic, solver.SolutionPool, solver.RNG)
                        
                        bestSol = iteratedGreedy.Run(bestInitialSol)

                        endTime = timeit.default_timer()

                        dict_iteration['Runtime'].append(endTime - startTime)
                        dict_iteration['Makespan'].append(bestSol.Makespan)

                        iteration += 1
    
    else:

        for numberJobsToRemove in list_numberJobsToRemove:
            for baseTemperature in list_baseTemperature:
                for maxIteration in list_maxIterations:
                
                    iteration = 0
                    while iteration < 3:
                            dict_iteration['Iteration'].append(iteration)
                            dict_iteration['LocalSearch'].append(str(LS_Algorithmus))
                            dict_iteration['NumberJobsToRemove'].append(numberJobsToRemove)
                            dict_iteration['baseTemperature'].append(baseTemperature)
                            dict_iteration['MaxIteration'].append(maxIteration)

                            seed = numberJobsToRemove*maxIteration*iteration
                            solver = Solver(data, seed)
                            localSearch = IterativeImprovement(data, 'BestImprovement', [])

                            localSearch.Initialize(solver.EvaluationLogic, solver.SolutionPool, solver.RNG)
                            iteratedGreedy = IteratedGreedy(
                            data,
                            numberJobsToRemove = numberJobsToRemove,
                            baseTemperature = baseTemperature,
                            maxIterations = maxIteration,
                            localSearchAlgorithm = localSearch)

                            bestInitialSol = solver.ConstructionPhase('NEH')

                            startTime = timeit.default_timer()
                            iteratedGreedy.Initialize(solver.EvaluationLogic, solver.SolutionPool, solver.RNG)
                            bestSol = iteratedGreedy.Run(bestInitialSol)
                    
                            endTime = timeit.default_timer()

                            dict_iteration['Runtime'].append(endTime - startTime)
                            dict_iteration['Makespan'].append(bestSol.Makespan)

                            iteration += 1

df = pd.DataFrame(dict_iteration, columns=['Iteration', 'LocalSearch', 'NumberJobsToRemove', 'baseTemperature', 'MaxIteration', 'Makespan', 'Runtime'])


df
                        



Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
Generating an initial solution according to NEH.
Constructive solution found.
The permutation [14, 5, 12, 10, 17, 8, 4, 11, 2, 15, 1, 18, 0, 16, 13, 19, 7, 3, 9, 6] results in a Makespan of 1665
New best solution in iteration 4: The permutation [12, 14, 10, 8, 16, 17, 0, 2, 13, 18, 1, 11, 19, 7, 3, 9, 6, 15, 5, 4] results in a Makespan of 1577
New best solution in iteration 5: The permutation [12, 14, 10, 8, 16,

,Iteration,LocalSearch,NumberJobsToRemove,baseTemperature,MaxIteration,Makespan,Runtime
0,0,Insertion,2,0.5,1,1595,2.312957
1,1,Insertion,2,0.5,1,1595,1.239241
2,2,Insertion,2,0.5,1,1595,1.416979
3,0,Insertion,2,0.5,10,1568,4.290261
4,1,Insertion,2,0.5,10,1569,3.761497
...,...,...,...,...,...,...,...
103,1,TaillardInsertion,4,1.0,1,1586,0.113687
104,2,TaillardInsertion,4,1.0,1,1583,0.136389
105,0,TaillardInsertion,4,1.0,10,1548,0.657254
106,1,TaillardInsertion,4,1.0,10,1561,0.504731


In [2]:
df_2 = df.groupby(['LocalSearch', 'NumberJobsToRemove', 'baseTemperature', 'MaxIteration']).mean()[['Makespan', 'Runtime']]
df_2

Makespan  \
LocalSearch       NumberJobsToRemove baseTemperature MaxIteration                
Insertion         2                  0.5             1             1595.000000   
                                                     10            1568.666667   
                                     1.0             1             1595.000000   
                                                     10            1568.666667   
                  3                  0.5             1             1586.333333   
                                                     10            1553.333333   
                                     1.0             1             1586.333333   
                                                     10            1553.333333   
                  4                  0.5             1             1580.000000   
                                                     10            1550.333333   
                                     1.0             1             1580.000000   
                                                     10            1550.333333   
None              2                  0.5             1             1665.000000   
                                                     10            1619.666667   
                                     1.0             1             1665.000000   
                                                     10            1619.666667   
                  3                  0.5             1             1659.333333   
                                                     10            1614.333333   
                                     1.0             1             1659.333333   
                                                     10            1614.333333   
                  4                  0.5             1             1644.333333   
                                                     10            1612.666667   
                                     1.0             1             1644.333333   
                                                     10            1615.000000   
TaillardInsertion 2                  0.5             1             1595.000000   
                                                     10            1568.666667   
                                     1.0             1             1595.000000   
                                                     10            1568.666667   
                  3                  0.5             1             1586.333333   
                                                     10            1553.333333   
                                     1.0             1             1586.333333   
                                                     10            1553.333333   
                  4                  0.5             1             1580.000000   
                                                     10            1550.333333   
                                     1.0             1             1580.000000   
                                                     10            1550.333333   

                                                                     Runtime  
LocalSearch       NumberJobsToRemove baseTemperature MaxIteration             
Insertion         2                  0.5             1              1.656393  
                                                     10             4.402137  
                                     1.0             1              1.296512  
                                                     10             4.317621  
                  3                  0.5             1              1.542703  
                                                     10             6.027122  
                                     1.0             1              1.578175  
                                                     10            10.228568  
                  4                  0.5             1              3.176961  
                                                     10            14.317617  
     

Mögliches erwartetes Ergebnis in der Ausgabe:

![Ausgabe](Ausgabe.PNG)
![Ausgabe](Ausgabe.PNG)